## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. **The code in this practical will not work with Python 2 unless you modify it.** If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [1]:
# -*- coding: utf-8 -*-
# NOTE:
# The original code has been modified from python3 to python2 as I installed anaconda with python2 before,
# so the following code should be run on python2 instead of python3

import numpy as np
import os
from random import shuffle
import re

In [2]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [3]:
#import urllib.request
import urllib
import zipfile
import lxml.etree

In [4]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [5]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()')) #find element named "content" by XPath and extract all texts 
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [6]:
#show uncleared texts
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

u' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by <u>removing all parenthesized strings using a regex</u>:

In [7]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text) 

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [8]:
#show texts after removing parenthesis
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

u"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to <u>remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`",</u> as shown in this example. Of course, this is an imperfect heuristic. 

In [9]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

[u"Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 u'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 u' Both are necessary, but it can be too much of a good thing',
 u'Consider Facit',
 u" I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? <u>We'll take the simplest naive approach of splitting on spaces.</u> Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [10]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split() #[^] resereves a-z and 0-9
    sentences_ted.append(tokens)

Two sample processed sentences:

In [11]:
len(sentences_ted)

266694

In [12]:
print(sentences_ted[0])
print(sentences_ted[1])

[u'here', u'are', u'two', u'reasons', u'companies', u'fail', u'they', u'only', u'do', u'more', u'of', u'the', u'same', u'or', u'they', u'only', u'do', u'what', u's', u'new']
[u'to', u'me', u'the', u'real', u'real', u'solution', u'to', u'quality', u'growth', u'is', u'figuring', u'out', u'the', u'balance', u'between', u'two', u'activities', u'exploration', u'and', u'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [4]:
from collections import Counter

In [13]:
c = Counter()
for sent in sentences_ted:
    for word in sent:
        c[word] += 1
words_top_ted = [tuple[0] for tuple in c.most_common(1000)] #top 1000 words
counts_ted_top1000 = [tuple[1] for tuple in c.most_common(1000)] #top 1000 counts of word
#print counts_ted_top1000

Plot distribution of top-1000 words

In [14]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [5]:
from gensim.models import Word2Vec

D:\ProgramData\Anaconda2\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [16]:
if not os.path.isfile('./tmp/myWord2Vec_ted'):
    model_ted = Word2Vec(sentences_ted, size=100, min_count=10)
    model_ted.save('./tmp/myWord2Vec_ted') #save for future load
else:
    model_ted = Word2Vec.load('./tmp/myWord2Vec_ted')

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [17]:
model_ted.most_similar("man")

[(u'woman', 0.8315613269805908),
 (u'guy', 0.8146558403968811),
 (u'lady', 0.7761974930763245),
 (u'boy', 0.7529250383377075),
 (u'girl', 0.7514427900314331),
 (u'soldier', 0.7263503074645996),
 (u'gentleman', 0.7179666757583618),
 (u'kid', 0.689196765422821),
 (u'rabbi', 0.6730810403823853),
 (u'philosopher', 0.6728865504264832)]

In [18]:
model_ted.most_similar("computer")

[(u'machine', 0.7396014928817749),
 (u'software', 0.7283523082733154),
 (u'robot', 0.7240867614746094),
 (u'device', 0.6887626647949219),
 (u'3d', 0.6630188822746277),
 (u'program', 0.6493382453918457),
 (u'video', 0.6408596038818359),
 (u'camera', 0.6207435727119446),
 (u'printer', 0.616905152797699),
 (u'simulation', 0.6167727708816528)]

In [19]:
print model_ted.most_similar_cosmul(positive=['woman', 'king'], negative=['man']) #cosine similarity

print model_ted.wv.doesnt_match("cat dog pig document".split()) #find the one that most likely dismatch

[(u'president', 0.9202700257301331), (u'french', 0.905616819858551), (u'poet', 0.9025776982307434), (u'mary', 0.9009045362472534), (u'obama', 0.8999260663986206), (u'luther', 0.8985649347305298), (u'queen', 0.8975723385810852), (u'james', 0.8958981037139893), (u'william', 0.89044588804245), (u'named', 0.8855022192001343)]
document


**Bonus: manually retrieve two words and use numpy to compute their cosine distance.**

In [43]:
print model_ted.wv.similarity('woman','man') #similarity using gensim using TED data

word1 = model_ted.wv['woman'] #output the vector of 'woman' and 'man'
word2 = model_ted.wv['man']

#'np.linalg.norm' usage: http://blog.csdn.net/lanchunhui/article/details/51004387
cosine = np.dot(word1, word2) / (np.linalg.norm(word1) * np.linalg.norm(word2)) 
print cosine 


0.831561443024
0.831561


#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [6]:
from sklearn.manifold import TSNE

In [21]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]
#print words_top_vec_ted[0] #check vector for 'the'

In [22]:
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [23]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted)) #'names':which word the point represents

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

---

### Part 5: Wiki Learnt Representations

Download dataset

In [24]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [7]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    #input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian
    raw_text = z.open('wikitext-103-raw/wiki.train.raw').read()
    input_text = raw_text.decode('utf-8')
    del raw_text

In [9]:
from pprint import pprint
a = input_text[0:1840]
pprint(a.split('\n'))

[u' ',
 u' = Valkyria Chronicles III = ',
 u' ',
 u' Senj\u014d no Valkyria 3 : Unrecorded Chronicles ( Japanese : \u6226\u5834\u306e\u30f4\u30a1\u30eb\u30ad\u30e5\u30ea\u30a23 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . ',
 u" The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also under

Preprocess sentences (note that it's important to remove small sentences for performance)

In [11]:
sentences_strings_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_strings_wiki.extend(s)
pprint(sentences_strings_wiki[:5])

[u' = Valkyria Chronicles III = ',
 u' Senj\u014d no Valkyria 3 : Unrecorded Chronicles ( Japanese : \u6226\u5834\u306e\u30f4\u30a1\u30eb\u30ad\u30e5\u30ea\u30a23 , lit ',
 u' Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media',
 u'Vision for the PlayStation Portable ',
 u' Released in January 2011 in Japan , it is the third game in the Valkyria series ']


In [12]:
pprint(len(sentences_strings_wiki))
pprint(sentences_strings_wiki[1005])
tmp = re.sub("[^a-z]", " ", sentences_strings_wiki[1005].lower())
tmp2 = re.sub(r'\([^)]*\)', '', tmp)
pprint(tmp)
pprint(tmp2)

4267112
u'@ 5 inches ( 191 mm ) of pine , alternating in direction '
u'    inches       mm   of pine   alternating in direction '
u'    inches       mm   of pine   alternating in direction '


In [13]:
sentences_wiki = []
for s_i in range(len(sentences_strings_wiki)):
    tokens = re.sub("[^a-z]", " ", sentences_strings_wiki[s_i].lower()).split() #only reserve 26 letters
    # ATTENTION: each sentence must be split into word list, whici is not finished by the original given code
    ###sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_strings_wiki[s_i]).split() #remove parenthesis
    sentences_wiki.append(tokens)
del input_text
del sentences_strings_wiki

In [18]:
print sentences_wiki[0]
print sentences_wiki[12000]
print len(sentences_wiki)

[u'valkyria', u'chronicles', u'iii']
[u'they', u'claimed', u'two', u'direct', u'hits', u'without', u'loss', u'despite', u'anti', u'aircraft', u'fire', u'from', u'the', u'cruiser', u'and', u'other', u'ships', u'and', u'frontal', u'attacks', u'by', u'enemy', u'fighters']
4267112


In [19]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

4267112
853422


In [26]:
print sentences_wiki[0]
print sentences_wiki[12000] #after shuffled, the list is different from the same code output above
print len(sentences_wiki)

[u'the', u'top', u'four', u'places', u'in', u'the', u'atp', u'rankings', u'for', u'all', u'but', u'weeks', u'from', u'september', u'january']
[u'legacy']
853422


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [28]:
cc = Counter()
for sent in sentences_wiki:
    for word in sent:
        cc[word] += 1
words_top_wiki = [tuple[0] for tuple in cc.most_common(1000)] #top 1000 words
#counts_wiki_top1000 = [tuple[1] for tuple in cc.most_common(1000)] #top 1000 counts of word

In [32]:
pprint(words_top_wiki)

[u'the',
 u'of',
 u'and',
 u'in',
 u'to',
 u'a',
 u'was',
 u's',
 u'on',
 u'as',
 u'for',
 u'that',
 u'with',
 u'by',
 u'is',
 u'his',
 u'at',
 u'he',
 u'it',
 u'from',
 u'were',
 u'an',
 u'had',
 u'which',
 u'be',
 u'this',
 u'but',
 u'are',
 u'first',
 u'not',
 u'their',
 u'after',
 u'one',
 u'her',
 u'also',
 u'its',
 u'they',
 u'two',
 u'or',
 u'have',
 u'who',
 u'new',
 u'been',
 u'has',
 u'she',
 u'when',
 u'time',
 u'during',
 u'would',
 u'other',
 u'into',
 u'all',
 u'i',
 u'more',
 u'while',
 u'over',
 u'game',
 u'only',
 u'him',
 u'later',
 u'up',
 u'most',
 u'three',
 u'about',
 u'out',
 u'th',
 u'between',
 u'than',
 u'there',
 u'some',
 u'film',
 u'may',
 u'year',
 u'before',
 u'made',
 u'no',
 u'such',
 u'where',
 u'second',
 u'season',
 u'war',
 u'years',
 u'world',
 u'both',
 u'then',
 u'however',
 u'used',
 u'through',
 u'them',
 u'being',
 u'these',
 u'series',
 u'against',
 u'song',
 u'm',
 u'album',
 u'many',
 u'team',
 u'city',
 u'part',
 u'number',
 u'north',
 u's

In [33]:
if not os.path.isfile('./tmp/myWord2Vec_wiki'):
    model_wiki = Word2Vec(sentences_wiki, size=100, min_count=10)
    model_wiki.save('./tmp/myWord2Vec_wiki')
else:
    model_wiki = Word2Vec.load('./tmp/myWord2Vec_wiki')


Compare model_wiki with model_ted.

In [41]:
print model_wiki.wv.similarity('woman','man') #similarity using gensim using wiki data
model_wiki.most_similar("dog")

0.721432784795


[(u'cat', 0.8147861361503601),
 (u'shoes', 0.710137665271759),
 (u'wolf', 0.6841351389884949),
 (u'rabbit', 0.6737985610961914),
 (u'pig', 0.658912718296051),
 (u'wig', 0.6572047472000122),
 (u'pants', 0.6544460654258728),
 (u'killer', 0.6537215113639832),
 (u'kid', 0.6463688611984253),
 (u'broom', 0.6442916393280029)]

#### t-SNE visualization

In [34]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [35]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)